In [1]:
%load_ext lab_black

In [17]:
import pandas as pd
from zipfile import ZipFile

In [3]:
# take a look at the data

file_companies_2005_Q1 = "../data/raw/company_2005_Q1.zip"
file_companies_2011_Q4 = "../data/raw/company_2011_Q4.zip"

myzip = ZipFile(file_companies_2005_Q1)
#     print(myzip.namelist())
filestream_2005 = myzip.open("company.idx")
myzip.close()

myzip = ZipFile(file_companies_2011_Q4)
filestream_2011 = myzip.open("company.idx")
myzip.close()

In [4]:
# load company list data
df_2005 = pd.read_csv(
    filestream_2005,
    sep=r" [ ]+",
    header=4,
    skiprows=[8],
    error_bad_lines=False,
    warn_bad_lines=False,
)
df_2005.info()

/Users/navy/anaconda3/envs/propulsion/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
/Users/navy/anaconda3/envs/propulsion/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/navy/anaconda3/envs/propulsion/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314826 entries, 0 to 314825
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Company Name  314826 non-null  object 
 1   Form Type     314825 non-null  object 
 2   CIK           314825 non-null  float64
 3   Date Filed    314825 non-null  object 
 4   File Name     314825 non-null  object 
dtypes: float64(1), object(4)
memory usage: 12.0+ MB


In [5]:
# load company list data
df_2011 = pd.read_csv(
    filestream_2011,
    sep=r" [ ]+",
    header=4,
    skiprows=[9],
    error_bad_lines=False,
    warn_bad_lines=False,
    encoding_errors="replace",
)
df_2011.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202470 entries, 0 to 202469
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Company Name  202470 non-null  object
 1   Form Type     202470 non-null  object
 2   CIK           202470 non-null  int64 
 3   Date Filed    202470 non-null  object
 4   File Name     202470 non-null  object
dtypes: int64(1), object(4)
memory usage: 7.7+ MB


In [6]:
# cleaning


def to_int(x):
    try:
        return int(x)
    except (pd.errors.IntCastingNaNError, ValueError):
        return -1


# 2005
df_2005["CIK"] = df_2005["CIK"].apply(lambda x: to_int(x))
df_2005["Date_filed"] = pd.to_datetime(df_2005["Date Filed"])
df_2005["Form Type"] = df_2005["Form Type"].astype("category")

# 2011
df_2011["Date_filed"] = pd.to_datetime(df_2011["Date Filed"])
df_2011["Form Type"] = df_2011["Form Type"].astype("category")

df_2005.to_csv("../data/processed/companies_2005_Q1.csv")
df_2011.to_csv("../data/processed/companies_2011_Q4.csv")

In [80]:
# select only 10-Q fillings

# some companies didn't fill 10-Q (why??), for instance AMAZON INC COM
df_2005_10Q = df_2005[df_2005["Form Type"].isin(["10-Q", "10-K"])]
df_2011_10Q = df_2011[df_2011["Form Type"].isin(["10-Q", "10-K"])]
print(df_2005_10Q.shape, df_2011_10Q.shape)

df_2005_10Q.drop_duplicates("Company Name", inplace=True)
df_2011_10Q.drop_duplicates("Company Name", inplace=True)
print(df_2005_10Q.shape, df_2011_10Q.shape)

(8272, 6) (8461, 6)
(7984, 6) (7984, 6)


In [81]:
counts = df_2005_10Q.value_counts("Company Name")
print(sum(counts > 1) / len(counts))
counts

0.0


Company Name
1 800 CONTACTS INC                      1
PALL CORP                               1
PANHANDLE ROYALTY CO                    1
PANHANDLE EASTERN PIPE LINE CO LP       1
PANERA BREAD CO                         1
                                       ..
FIRST NATIONAL LINCOLN CORP /ME/        1
FIRST NATIONAL FUNDING LLC              1
FIRST NATIONAL CORP /VA/                1
FIRST NATIONAL COMMUNITY BANCORP INC    1
streetTRACKS GOLD TRUST                 1
Length: 7984, dtype: int64

In [82]:
counts = df_2011_10Q.value_counts("Company Name")
print(sum(counts > 1) / len(counts))
counts

0.0


Company Name
'mktg, inc.'                  1
Ocean Shore Holding Co.       1
OmniaLuo, Inc.                1
OmniAmerican Bancorp, Inc.    1
Omega Flex, Inc.              1
                             ..
Enhance Skin Products Inc     1
Engraving Masters, Inc.       1
Engchow Education Corp        1
Enertopia Corp.               1
vitaminspice                  1
Length: 7984, dtype: int64

In [83]:
# What's happening with this company... ?
df_2011_10Q[df_2011_10Q["Company Name"] == "Grupo International Inc"]

,Company Name,Form Type,CIK,Date Filed,File Name,Date_filed
81954,Grupo International Inc,10-K,1082576,2011-10-19,edgar/data/1082576/0001269678-11-000230.txt,2011-10-19


In [84]:
# Select only companies (a bit rough...)

is_company = ["INC", "Inc", "CORP", "LTD", "Corp", "CO", "inc"]
df_2005_10Q_inc = df_2005_10Q[
    df_2005_10Q["Company Name"].str.contains("|".join(is_company))
]
df_2011_10Q_inc = df_2011_10Q[
    df_2011_10Q["Company Name"].str.contains("|".join(is_company))
]

In [85]:
# Check the remaining entries

df_2005_10Q_fund = df_2005_10Q[
    ~df_2005_10Q["Company Name"].str.contains("|".join(is_company))
]
df_2011_10Q_fund = df_2011_10Q[
    ~df_2011_10Q["Company Name"].str.contains("|".join(is_company))
]
df_2005_10Q_fund
df_2011_10Q_fund

,Company Name,Form Type,CIK,Date Filed,File Name,Date_filed
452,99 CENTS ONLY STORES,10-Q,1011290,2011-11-10,edgar/data/1011290/0001140361-11-052840.txt,2011-11-10
585,AAA CAPITAL ENERGY FUND L.P.,10-Q,1057051,2011-11-14,edgar/data/1057051/0000950123-11-098443.txt,2011-11-14
587,AAA CAPITAL ENERGY FUND L.P. II,10-Q,1227268,2011-11-14,edgar/data/1227268/0000950123-11-098383.txt,2011-11-14
704,ABBOTT LABORATORIES,10-Q,1800,2011-11-04,edgar/data/1800/0001104659-11-061064.txt,2011-11-04
1059,ACADIA REALTY TRUST,10-Q,899629,2011-11-02,edgar/data/899629/0001445305-11-003166.txt,2011-11-02
...,...,...,...,...,...,...
201422,"Zhong Wen International Holding Co., Ltd.",10-Q,1494502,2011-11-14,edgar/data/1494502/0001144204-11-063700.txt,2011-11-14
202180,iShares S&P GSCI Commodity-Indexed Investing P...,10-Q,1346616,2011-11-08,edgar/data/1346616/0001193125-11-302344.txt,2011-11-08
202186,iShares S&P GSCI Commodity-Indexed Trust,10-Q,1332174,2011-11-08,edgar/data/1332174/0001193125-11-302344.txt,2011-11-08
202192,iShares Silver Trust,10-Q,1330568,2011-11-08,edgar/data/1330568/0001193125-11-302064.txt,2011-11-08


In [86]:
# df_2005_10Q_inc.drop_duplicates("Company Name", inplace=True)
# df_2011_10Q_inc.drop_duplicates("Company Name", inplace=True)
print(df_2005_10Q_inc.shape, df_2011_10Q_inc.shape)

(6412, 6) (7185, 6)


In [87]:
# merge to see what companies listed in 2005 are still there in 2011

df_merge = pd.merge(
    df_2005_10Q_inc,
    df_2011_10Q_inc,
    on="Company Name",
    how="inner",
    validate="one_to_one",
)
print(df_merge.shape)
print(len(df_2011_10Q_inc) - len(df_merge))

(2691, 11)
4494


Out of 1397 companies listed in 2005, 418 remain in 2011.<br>
Meanwhile, 6259 (!!) companies went public after Q1 2005.

In [88]:
# save final files to CSV

df_2005_10Q_inc.to_csv("../data/processed/companies_2005_10Q.csv")
df_2011_10Q_inc.to_csv("../data/processed/companies_2011_10Q.csv")

In [93]:
cik_counts_2005 = df_2005_10Q_inc.value_counts("CIK")
cik_counts_2011 = df_2011_10Q_inc.value_counts("CIK")

cik_counts_2005

CIK
20         1
1053439    1
1054374    1
1054298    1
1054290    1
          ..
825788     1
825692     1
825542     1
825521     1
1319549    1
Length: 6412, dtype: int64

In [94]:
cik_counts_2011

CIK
1500365    2
1002822    2
1750       1
1288403    1
1289236    1
          ..
901491     1
901219     1
900393     1
900349     1
1530796    1
Length: 7183, dtype: int64

In [95]:
# try merging on CIK instead

df_merge_2 = pd.merge(
    df_2005_10Q_inc,
    df_2011_10Q_inc,
    on="CIK",
    how="inner",
    #     validate="one_to_one",
)
df_merge_2.shape

(3150, 11)

In [96]:
cik_counts_2005 = df_2005_10Q_inc.value_counts("CIK")
cik_counts_2011 = df_2011_10Q_inc.value_counts("CIK")

print(cik_counts_2005[cik_counts_2005 > 1])
print(cik_counts_2011[cik_counts_2011 > 1])

# no duplicate cik in either dataset

Series([], dtype: int64)
CIK
1500365    2
1002822    2
dtype: int64


In [97]:
cik_survived = set(df_merge_2["CIK"])
len(cik_survived)

3150

In [98]:
cik_died = df_2005_10Q_inc.loc[~df_2005_10Q_inc["CIK"].isin(cik_survived), "CIK"]
cik_died = set(cik_died)
len(cik_died)

3262

In [99]:
companies_survived = df_2005_10Q_inc.loc[
    df_2005_10Q_inc["CIK"].isin(cik_survived), ["Company Name", "CIK"]
].reset_index(drop=True)
companies_died = df_2005_10Q_inc.loc[
    df_2005_10Q_inc["CIK"].isin(cik_died), ["Company Name", "CIK"]
].reset_index(drop=True)

# upper-case all company names to correspond to Mevluet dataset
companies_survived["Company Name"] = companies_survived["Company Name"].str.upper()
companies_died["Company Name"] = companies_died["Company Name"].str.upper()

In [110]:
# FIXME
companies_died.drop_duplicates("Company Name", inplace=True)

In [106]:
companies_survived.head()

,Company Name,CIK
0,1 800 FLOWERS COM INC,1084869
1,1ST CONSTITUTION BANCORP,1141807
2,1ST SOURCE CORP,34782
3,21ST CENTURY HOLDING CO,1069996
4,3D SYSTEMS CORP,910638


In [101]:
companies_died.head()

,Company Name,CIK
0,1 800 CONTACTS INC,1050122
1,1ST STATE BANCORP INC,1074078
2,24/7 REAL MEDIA INC,1062195
3,3CI COMPLETE COMPLIANCE CORP,883787
4,3COM CORP,738076


In [111]:
companies_survived.to_csv("../data/processed/companies_survived.csv")
companies_died.to_csv("../data/processed/companies_died.csv")

In [66]:
df_2005.head()

,Company Name,Form Type,CIK,Date Filed,File Name,Date_filed
0,----------------------------------------------...,NaN,-1,None,None,NaT
1,033 ASSET MANAGEMENT LLC /,13F-HR,1114831,2005-02-04,edgar/data/1114831/0000950136-05-000593.txt,2005-02-04
2,033 ASSET MANAGEMENT LLC /,SC 13D,1114831,2005-02-09,edgar/data/1114831/0000950136-05-000662.txt,2005-02-09
3,033 ASSET MANAGEMENT LLC /,SC 13D/A,1114831,2005-01-26,edgar/data/1114831/0000950136-05-000374.txt,2005-01-26
4,033 ASSET MANAGEMENT LLC /,SC 13D/A,1114831,2005-02-04,edgar/data/1114831/0000950136-05-000613.txt,2005-02-04


In [119]:
companies_died[companies_died["Company Name"].str.contains("ORACLE")]

,Company Name,CIK
2183,ORACLE CORP /DE/,777676


In [121]:
df_2005[df_2005["Company Name"].str.contains("ORACLE")].head()

,Company Name,Form Type,CIK,Date Filed,File Name,Date_filed
174250,MANALAPAN ORACLE EAGLE FUND LP,REGDEX,1134424,2005-01-24,edgar/data/1134424/9999999997-05-002904.txt,2005-01-24
211400,ORACLE CORP /DE/,10-Q,777676,2005-03-25,edgar/data/777676/0000950134-05-006028.txt,2005-03-25
211401,ORACLE CORP /DE/,3,777676,2005-01-03,edgar/data/777676/0000950103-05-000010.txt,2005-01-03
211402,ORACLE CORP /DE/,4,777676,2005-01-03,edgar/data/777676/0000950103-05-000011.txt,2005-01-03
211403,ORACLE CORP /DE/,4,777676,2005-01-03,edgar/data/777676/0001093287-05-000002.txt,2005-01-03


In [122]:
df_2011[df_2011["Company Name"].str.contains("ORACLE")].head()

,Company Name,Form Type,CIK,Date Filed,File Name,Date_filed
136358,ORACLE CORP,10-Q,1341439,2011-12-23,edgar/data/1341439/0001193125-11-351954.txt,2011-12-23
136359,ORACLE CORP,4,1341439,2011-10-18,edgar/data/1341439/0001127602-11-027473.txt,2011-10-18
136360,ORACLE CORP,4,1341439,2011-11-02,edgar/data/1341439/0001127602-11-028328.txt,2011-11-02
136361,ORACLE CORP,4,1341439,2011-11-04,edgar/data/1341439/0001127602-11-028698.txt,2011-11-04
136362,ORACLE CORP,4,1341439,2011-11-15,edgar/data/1341439/0001127602-11-029678.txt,2011-11-15
